## Set-up

In [1]:
# Upgrade pip
!pip install --upgrade pip

In [2]:
!pip install --user google-cloud-bigquery==3.4.1
!pip install --user pandas
!pip install google-cloud-bigquery
!pip install google-cloud-bigquery-storage
!pip install google-cloud-storage
!pip install pyarrow
!pip install db-dtypes
!pip install tqdm
!pip install matplotlib
!pip install ipywidgets
!pip install ipywidgets
!pip install google-cloud-aiplatform
!jupyter nbextension enable --py widgetsnbextension

  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/33/8a/e0bb96ad439a3cc11671a4f5fb1ff2f0d3a9993e325d82859ba75cfebd77/google_cloud_aiplatform-1.32.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-resource-manager<3.0.0dev,>=1.3.3 from https://files.pythonhosted.org/packages/d2/f3/e7d1848edf2f4714d8925e6c749464c190e696b3c85264441c2b3787ae72/google_cloud_resource_manager-1.10.3-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 5.5 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
         

In [3]:
from google.cloud import bigquery
from google.cloud import storage
from tqdm import tqdm
import matplotlib.pyplot as plt
import ipywidgets
from google.cloud import aiplatform

In [12]:
!PROJECT_ID=$(gcloud config get-value project)
PROJECT_ID = "bqml-sandbox-396011"
VERTEX_AI_LOCATION = 'europe-west4'

In [4]:
%load_ext google.cloud.bigquery

In [13]:
aiplatform.init(project=PROJECT_ID, location=VERTEX_AI_LOCATION)

## Owerview

There are four ways to export BigQueryMl models:
1. by using the Google Cloud Console,
2. by using `EXPORT MODEL` statement,
3. by using `bq extract` command,
4. Using API or Client Library.

Most of the time the model is saved by default as `TensorfFlow SavedModel`

In [5]:
# list all models
!bq ls -m --format=pretty $PROJECT_ID:BQ_ML_ID


Welcome to BigQuery! This script will walk you through the 
process of initializing your .bigqueryrc configuration file.

First, we need to set up your credentials if they do not 
already exist.

Setting project_id bqml-sandbox-396011 as the default.

BigQuery configuration complete! Type "bq" to get started.

+--------------------------+--------------------------------+--------+-----------------+
|            Id            |           Model Type           | Labels |  Creation Time  |
+--------------------------+--------------------------------+--------+-----------------+
| BASE_LOGISTIC_REGRESSION | LOGISTIC_REGRESSION            |        | 10 Sep 08:38:52 |
| DNN                      | DNN_LINEAR_COMBINED_CLASSIFIER |        | 10 Sep 11:43:48 |
+--------------------------+--------------------------------+--------+-----------------+


In [18]:
# Create gcs bucket to store models

project_id = "bqml-sandbox-396011"
bucket_name = "bq-ml-store"
default_storage_class = "STANDARD" 

# Initialize the client
client = storage.Client(project=project_id)

# Create the bucket with the specified default storage class
bucket = client.bucket(bucket_name)
bucket.location = "EU"
bucket.storage_class = default_storage_class
    # Try to create the bucket (it will raise an error if it already exists)
try:
    bucket.create()
    print(f"Bucket '{bucket_name}' created with default storage class '{default_storage_class}'.")
except Exception as e:
    print(f"Error creating bucket: {e}")


/tmp/ipykernel_5043/2528192649.py:12: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = "EU"


Bucket 'bq-ml-store' created with default storage class 'STANDARD'.


In [19]:
!bq extract --model 'BQ_ML_ID.BASE_LOGISTIC_REGRESSION' gs://bq-ml-store/base-logistic-regression

Waiting on bqjob_r53868589930e1046_0000018a7f0057a2_1 ... (33s) Current status: DONE   


In [21]:
%%bigquery

 EXPORT MODEL `BQ_ML_ID.DNN`
 OPTIONS(URI = 'gs://bq-ml-store/dnn')
 

Query is running:   0%|          |

""


## Register the model in Vertex AI

In [25]:
%%bigquery
ALTER MODEL BQ_ML_ID.BASE_LOGISTIC_REGRESSION SET OPTIONS (vertex_ai_model_id="base_logistic_regression");

Query is running:   0%|          |

""


## Deploying Model in Vertex AI

In [15]:
# create an endpoint

endpoint = aiplatform.Endpoint.create(
        display_name= "base_logistic_regression",
        project= PROJECT_ID,
        location= VERTEX_AI_LOCATION,
    )


Creating Endpoint
Create Endpoint backing LRO: projects/115333740492/locations/europe-west4/endpoints/8224984692508590080/operations/4462031765648703488
Endpoint created. Resource name: projects/115333740492/locations/europe-west4/endpoints/8224984692508590080
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/115333740492/locations/europe-west4/endpoints/8224984692508590080')


In [18]:
# deploy a model
model = aiplatform.Model(model_name = "base_logistic_regression")
model.deploy(
    endpoint = endpoint,
    deployed_model_display_name = "base_logistic_regression",
    traffic_percentage = 100, # only one model in the endpoint so it must be 100%
    machine_type = "n1-standard-2",
    min_replica_count = 1,
    max_replica_count = 4,
    accelerator_type = None ,
    accelerator_count = None ,
    sync=True,
    )

model.wait()

Deploying model to Endpoint : projects/115333740492/locations/europe-west4/endpoints/8224984692508590080
Deploy Endpoint model backing LRO: projects/115333740492/locations/europe-west4/endpoints/8224984692508590080/operations/3963821056870842368


Endpoint model deployed. Resource name: projects/115333740492/locations/europe-west4/endpoints/8224984692508590080


resource name: projects/115333740492/locations/europe-west4/endpoints/8224984692508590080

In [ ]:
## Sample prediction
